# Read in Data

In [1]:
import os
import pandas as pd
import numpy as np
import xgboost

from sklearn import metrics,preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, fbeta_score

from nltk.corpus import stopwords

import re

import matplotlib.pyplot as plt
%matplotlib inline

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_data = pd.read_csv("./input/train_v2.csv")
test_data = pd.read_csv("./input/test_v2.csv")

In [3]:
train_data.shape

(6027, 7)

In [4]:
test_data.shape

(3826, 6)

In [6]:
#v2: remove numbers and punctuations
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
train_data['text']=""
#train_data['title_clean']=""
for i in range(len(train_data['title'])):
        filename='raw_train_new/'+str(i+1)+'.txt'
        title=train_data['title'][i]
        title = re.sub(r"[,:.;@#\?\-\%\/!&$]+\ *", " ", title)
        title = ''.join(filter(whitelist.__contains__, title)).lower()
        title = ' '.join(title.split())
       # train_data['title_clean'][i]=title
        #add space to title
        title=title+" "
        if os.path.exists(filename):
            with open(filename, 'r',errors='ignore') as data_file:
                data=data_file.read().replace('\n', '')
                data = re.sub(r"[,:.;@#\?\-\%\/!&$]+\ *", " ", data)
                data = ''.join(filter(whitelist.__contains__, data)).lower()
                data=' '.join(data.split())
            train_data['text'][i]=title*10+data
        else:
            train_data['text'][i]=title*10

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
#v2 remove numbers and punctuations
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
test_data['text']=""
#test_data['title_clean']=""
for i in range(len(test_data['title'])):
        filename='raw_test_new/'+str(i+1)+'.txt'
        title=test_data['title'][i]
        title = re.sub(r"[,:.;@#\?\-\%\/!&$]+\ *", " ", title)
        title = ''.join(filter(whitelist.__contains__, title)).lower()
        title = ' '.join(title.split())
        #test_data['title_clean'][i]=title
        title=title+" "
        if os.path.exists(filename):
            with open(filename, 'r',errors='ignore') as data_file:
                data=data_file.read().replace('\n', '')
                data = re.sub(r"[,:.;@#\?\-\%\/!&$]+\ *", " ", data)
                data = ''.join(filter(whitelist.__contains__, data)).lower()
                data=' '.join(data.split())
            test_data['text'][i]=title*10 +data
        else:
            test_data['text'][i]=title*10

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [48]:
import sklearn
def train_model(classifier, feature_vector_train, label_train, feature_vector_valid, label_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label_train)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return sklearn.metrics.accuracy_score(predictions, label_valid)

Only use news title to train tfidf

In [8]:
total_text=pd.concat([train_data['title'],test_data['title']]).values

In [27]:
print(len(total_text))

9853


In [10]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=3500)
tfidf_vect_ngram.fit(total_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=3500, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

use news news title *10 + news content to do tfidf transform

In [11]:
train_tfidf_ngram =  tfidf_vect_ngram.transform(train_data["text"])
test_tfidf_ngram =  tfidf_vect_ngram.transform(test_data["text"])

In [12]:
classifier = xgboost.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=140)
accuracy = train_model(classifier, train_tfidf_ngram.tocsc(), train_data["category"], train_tfidf_ngram.tocsc(), train_data["category"])
print("XGB on training set ", accuracy)

XGB on training set  0.8319230131076821


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
prob=classifier.predict_proba(test_tfidf_ngram)

In [14]:
prob.shape

(3826, 5)

In [15]:
prob[:10]

array([[0.10037063, 0.02017497, 0.338339  , 0.02973611, 0.5113793 ],
       [0.08138281, 0.03217706, 0.22272417, 0.0276122 , 0.63610375],
       [0.05535575, 0.10685922, 0.24474008, 0.04040945, 0.5526355 ],
       [0.07628933, 0.05074055, 0.23462062, 0.04981647, 0.588533  ],
       [0.00455279, 0.00581613, 0.02667828, 0.00276371, 0.96018904],
       [0.08533074, 0.04119757, 0.17929082, 0.05783598, 0.6363449 ],
       [0.2278105 , 0.00968871, 0.38149652, 0.01776791, 0.3632364 ],
       [0.02956243, 0.02622815, 0.10435862, 0.01901004, 0.8208408 ],
       [0.09902731, 0.05111004, 0.35687304, 0.07238013, 0.42060953],
       [0.16721213, 0.08650713, 0.17862843, 0.06461759, 0.5030347 ]],
      dtype=float32)

In [24]:
THRESHOLD = 0.6
mask = np.amax(prob, axis=1) > THRESHOLD

In [25]:
mask

array([False,  True, False, ...,  True, False,  True])

In [26]:
 ysemi = np.argmax(prob,axis=1)

In [29]:
unique_elements, counts_elements = np.unique(ysemi, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1    2    3    4]
 [ 693  262  777  286 1808]]


In [53]:
final_clf = SVC(kernel='linear')
accuracy = train_model(final_clf,
                                  sp.vstack((train_tfidf_ngram, test_tfidf_ngram[mask])),
                                  np.concatenate((train_data["category"], ysemi[mask])), 
                                  train_tfidf_ngram, train_data["category"])
print(accuracy)

0.7340301974448316


In [58]:
result=final_clf.predict(test_tfidf_ngram)

In [59]:
test_data["category"] = result

In [60]:
out=pd.DataFrame(test_data,columns=['article_id','category'])
out.to_csv('prediction.csv',index=False)